<a href="https://colab.research.google.com/github/mateusragazzi/bug-helper/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Dependencies

In [ ]:
!pip install fastparquet
!pip install python-Levenshtein
#!pip install textblob
#!python -m textblob.download_corpora

     |████████████████████████████████| 122kB 8.5MB/s 
     |████████████████████████████████| 61kB 7.1MB/s 
  Created wheel for fastparquet: filename=fastparquet-0.5.0-cp37-cp37m-linux_x86_64.whl size=213353 sha256=cb9eb0f94cd5f28c641d449439e41baf34061db016ec2e35bcff8cde5e895054
  Stored in directory: /root/.cache/pip/wheels/97/0e/da/e2122965947ab2bfbc20e31a968e998fc8932f7fcb0ed78ad0
  Created wheel for thrift: filename=thrift-0.13.0-cp37-cp37m-linux_x86_64.whl size=348151 sha256=746741f239910004ec287f6061e3fe4492bbb5a2e6cbe1f25553007565d91266
  Stored in directory: /root/.cache/pip/wheels/02/a2/46/689ccfcf40155c23edc7cdbd9de488611c8fdf49ff34b1706e
Successfully built fastparquet thrift
     |████████████████████████████████| 51kB 4.5MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=149810 sha256=938991dea7749cefc241f55a81d9f23866c73d7863f000b3699f819d094de25f
  Stored in directory: /root/.cache/pip/wheels/b3/26/73/4b48503

##### Imports

In [ ]:
from fastparquet import ParquetFile
from google.colab import drive
from datetime import datetime
import nltk
import pandas as pd
import numpy as np
import string
import Levenshtein

from nltk.corpus import stopwords
nltk.download('tagsets')
nltk.download('stopwords')

stopwords = stopwords.words('english')

drive.mount('/content/drive')

[nltk_data] Downloading package tagsets to /root/nltk_data...
[nltk_data]   Unzipping help/tagsets.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
Mounted at /content/drive


### Main Code

#### Read `.parquet` file, converting it to pandas DataFrame

In [ ]:

parquetFile = ParquetFile('drive/MyDrive/Colab/issues.parquet', verify=True)
pandasDataFrame = parquetFile.to_pandas()

# Filter only bugs
bugDataFrame = pandasDataFrame[pandasDataFrame["issuetype"].str.contains("Bug", na=False)]
# Filter except bugs
baseDataFrame = pandasDataFrame[~pandasDataFrame["issuetype"].str.contains("Bug", na=False)]


In [ ]:
baseDataFrame.loc[2314123]['summary']

'When a website requests VR access, Firefox Nightly puts up a link to a non-existant page'

#### Get one item from bug to analise and clean phrase

In [ ]:
def clean_string(text):
  if (text is not None):
    text = ' '.join([word for word in text.split() if word not in stopwords])
    text = ''.join([word for word in text if word not in string.punctuation])
    text = text.lower()
    text = text.strip().replace("\n", " ").replace("\r", " ")
    return text
  else:
    return ""

# focusedItem = bugDataFrame.loc[202]['summary']
focusedItem = "When a website requests VR access, Firefox Nightly puts up a link to a non-existant page"
focusedItem = clean_string(focusedItem)

print(focusedItem)

when website requests vr access firefox nightly puts link nonexistant page


#### Search the closest item inside base dataframe

In [ ]:
minDistance = 0;
betterIndex = 0;

for index, row in baseDataFrame.iterrows():
  phrase = clean_string(row['summary'])
  distance = Levenshtein.distance(focusedItem, phrase)
  if (minDistance == 0):
    minDistance = distance
    betterIndex = index
  elif (distance < minDistance) :
    minDistance = distance
    betterIndex = index

print("better index: ", betterIndex)
print("min distance: ", minDistance)

better index:  2314124
min distance:  59


#### Printing the severity and days to resolved

In [ ]:
item = baseDataFrame.loc[betterIndex]

severity = item['severity']
created = item['created']
last_resolved = item['last_resolved']

resolvedTime = (str) ((last_resolved - created).days)

if (resolvedTime is None):
  resolvedTime = "0"

if (severity is None):
  severity = "unknown"

print("Your problem has a " + severity + " severity and will take " + resolvedTime + " day(s) to resolve")

Your problem has a normal severity and will take nan day(s) to resolve
